In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os 
import io
import zipfile
from zipfile import ZipFile
import pandas as pd
import requests
import tensorflow as tf

In [3]:
fantasy_zip = ZipFile('/content/drive/MyDrive/Colab Notebooks/[원본]이미지_사운드매칭_cycle50.zip')
cate_list = []
with ZipFile('/content/drive/MyDrive/Colab Notebooks/[원본]이미지_사운드매칭_cycle50.zip', 'r') as zipObj:
  zipInfo = zipObj.infolist()
  print(len(zipInfo))
  # print(zipInfo)
  for member in zipInfo:
    try:
      member.filename = member.filename.encode('cp437').decode('euc-kr')
      cate_list.append(member.filename.split("/")[3])
    except:
      pass



130247


In [4]:
len(cate_list)
len(set(cate_list))
result_cate=list(set(cate_list))

In [5]:
result_cate

['',
 '대패질',
 '괘종시계 작동',
 '에어컨 사용',
 '버스 벨',
 '프론트 벨',
 '키질',
 '탁구 치기',
 '스프링클러 동작',
 '세탁기',
 '탄산 따르기',
 '냉장고 사용',
 '먹 갈기',
 '썰기',
 '키보드 치기',
 '타자 치기',
 '캐리어 사용',
 '커피 내리기',
 '드라이어 사용',
 '불 지피기',
 '맷돌 갈기',
 '엿장수',
 '튀기기',
 '예초기 작업',
 '골프 치기',
 '석유풍로 사용',
 '청소하기',
 '콤바인 작업',
 '무치기',
 '전자레인지 사용',
 '화투치기',
 '차량 주행',
 '확독 갈기',
 '숫돌 갈기',
 '주전자 사용',
 '굽기',
 '경운기 작업',
 '볼링 치기',
 '압력밥솥 사용',
 '커피콩 갈기',
 '사찰 종',
 '러닝머신 운동하기',
 '양수기 동작',
 '따르기',
 '차량 공회전',
 '작두질',
 '면도하기(전동)',
 '절구질',
 '떡메질',
 '학교 차임벨',
 '다지기(고기)',
 '당구 치기',
 '전 부치기',
 '경고음',
 '초인종',
 '다듬이질',
 '횃불 지피기',
 '선풍기 사용',
 '안마하기',
 '안마하기(전동)',
 '면도하기',
 '식당 벨',
 '트랙터 작업',
 '다지기(채소)',
 '분무기 작업',
 '자전거 타기',
 '교회 종',
 '싸리비 질',
 '풍로 사용',
 '끓이기',
 '깎기',
 '선박 운항',
 '믹서기 갈기',
 '물레방아 돌기',
 '이발하기',
 '커피콩 볶기']

In [44]:
# 라벨 목록 조사

label_list = []
with ZipFile('/content/drive/MyDrive/Colab Notebooks/[원본]이미지_사운드매칭_cycle50.zip', 'r') as zipObj:
  zipInfo = zipObj.infolist()
  # print(zipInfo)
  for member in zipInfo:
    member.filename = member.filename.encode('cp437').decode('euc-kr')
    try:
      if (member.filename.split("/")[2]=="라벨링데이터") and (member.filename.split("/")[1]=="이미지"):
        tempbox = member.filename.split("/")[5].split("_")
        label_list.append(tempbox[0]+"_"+tempbox[1])
    except:
      pass
len(set(label_list))
label_list=list(set(label_list))

In [49]:
import json



filename_group = dict()



file1 = dict()

file1["1"] = result_cate[:int(len(result_cate)/7)]

file1["2"] = result_cate[int(len(result_cate)/7):int(len(result_cate)*(2/7))]

file1["3"] = result_cate[int(len(result_cate)*(2/7)):int(len(result_cate)*(3/7))]

file1["4"] = result_cate[int(len(result_cate)*(3/7)):int(len(result_cate)*(4/7))]

file1["5"] = result_cate[int(len(result_cate)*(4/7)):int(len(result_cate)*(5/7))]

file1["6"] = result_cate[int(len(result_cate)*(5/7)):int(len(result_cate)*(6/7))]

file1["7"] = result_cate[int(len(result_cate)*(6/7)):]


filename_group["file"] = file1

filename_group["labels"] = label_list




#json 파일로 저장

with open('/content/file_list.json', 'w') as make_file:

    json.dump(filename_group, make_file, indent="\t")

In [6]:
len(file1["1"])+len(file1["2"])+len(file1["3"])+len(file1["4"])+len(file1["5"])+len(file1["6"])+len(file1["7"])

76

### 분할 압축 풀기

In [4]:
# 학습순서 및 클래스 정보 json 불러오기

import json



with open('/content/file_list (1).json', 'r') as f:

    json_data = json.load(f)

sector=json.dumps(json_data, ensure_ascii = False)
sector=json.loads(sector)
print(sector["file"]["1"])
print(sector["file"]["2"])
label_list = sector["labels"]

['', '트랙터 작업', '다지기(고기)', '대패질', '깎기', '괘종시계 작동', '면도하기', '선박 운항', '식당 벨', '이발하기']
['다듬이질', '불 지피기', '절구질', '주전자 사용', '초인종', '안마하기(전동)', '스프링클러 동작', '러닝머신 운동하기', '전자레인지 사용', '교회 종', '다지기(채소)']


In [5]:
with ZipFile('/content/drive/MyDrive/Colab Notebooks/[원본]이미지_사운드매칭_cycle50.zip', 'r') as zipObj:
  zipInfo = zipObj.infolist()
  # print(zipInfo)
  for member in zipInfo:
    member.filename = member.filename.encode('cp437').decode('euc-kr')
    try:
      if member.filename.split("/")[1]=="이미지" and (member.filename.split("/")[3] in sector["file"]["1"]):
        zipObj.extract(member, '/content')
        # print(member.filename)
    except:
      zipObj.extract(member, '/content')
      # print(member.filename)

In [5]:
len(label_list)

154

In [47]:
label_list

['전자레인지 사용_전자레인지',
 '석유풍로 사용_석유풍로',
 '썰기_상추',
 '다지기(고기)_고기',
 '튀기기_파프리카',
 '경고음_구급차',
 '차량 공회전_승용차',
 '깎기_감자',
 '따르기_포도주',
 '썰기_사과',
 '안마하기(전동)_전신 안마기',
 '엿장수_가위질',
 '튀기기_상추',
 '청소하기_무선 진공청소기',
 '횃불 지피기_기름 횃불',
 '키질_키',
 '따르기_막걸리',
 '확독 갈기_확독',
 '차량 공회전_버스',
 '떡메질_떡메',
 '키보드 치기_노트북',
 '굽기_닭고기',
 '믹서기 갈기_곡물갈기',
 '탄산 따르기_맥주',
 '커피콩 갈기_수동 그라인더',
 '탁구 치기_탁구',
 '숫돌 갈기_숫돌',
 '스프링클러 동작_스프링쿨러',
 '세탁기_통돌이 세탁기',
 '커피콩 갈기_자동 그라인더',
 '썰기_파',
 '선풍기 사용_공업용 선풍기',
 '교회 종_교회 종',
 '키보드 치기_일반 키보드',
 '싸리비 질_싸리비',
 '이발하기_전동 이발기',
 '커피 내리기_자판기',
 '튀기기_고기',
 '경고음_철길 건널목',
 '믹서기 갈기_고기갈기',
 '압력밥솥 사용_일반 압력밥솥',
 '콤바인 작업_수확',
 '청소하기_유선 진공청소기',
 '당구 치기_당구',
 '양수기 동작_펌프',
 '차량 주행_승용차',
 '경운기 작업_주행',
 '전 부치기_호박',
 '썰기_참외',
 '불 지피기_모닥불',
 '풍로 사용_풍로',
 '절구질_절구',
 '안마하기(전동)_핸드 안마기',
 '전 부치기_파',
 '차량 주행_버스',
 '썰기_배추',
 '무치기_겉절이',
 '믹서기 갈기_과일갈기',
 '세탁기_드럼 세탁기',
 '화투치기_화투 치기',
 '끓이기_찌개',
 '괘종시계 작동_괘종시계',
 '전 부치기_김치',
 '썰기_호박',
 '물레방아 돌기_물레방아',
 '커피 내리기_에스프레소 머신',
 '볼링 치기_볼링',
 '차량 주행_SUV',
 '다지기(채소)_양파',
 '면도하기(전동)_전기 면도기

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
from pathlib import Path
import base64
import cv2
import matplotlib.pyplot as plt
import json 
from tqdm.notebook import tqdm 
import scipy 
from joblib import Parallel , delayed

In [7]:
sample = cv2.imread(f'/content/[원본]이미지_사운드매칭_cycle50/이미지/원천데이터/불 지피기/모닥불/불 지피기_모닥불_Im10141370.jpg')
print(sample.shape)
sample = cv2.rectangle(sample,(1917,577),(1917+1072,577+922),(0,0,0),10)
plt.imshow(sample)

AttributeError: ignored

In [7]:
import os
 
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
createFolder('/content/data')
createFolder('/content/data/images')
createFolder('/content/data/labels')


In [9]:
# 전처리 및 파일 생성
# 폴더 이름 바꾸기!
import cv2
import json
from glob import glob
def padding(img, set_size):

    try:
        h,w,c = img.shape
        H = h
    except:
        print('파일을 확인후 다시 시작하세요.')
        raise

    if h < w:
        new_width = set_size
        new_height = int(new_width * (h/w))
    else:
        new_height = set_size
        new_width = int(new_height * (w/h))

    if max(h, w) < set_size:
        img = cv2.resize(img, (new_width, new_height), cv2.INTER_CUBIC)
    else:
        img = cv2.resize(img, (new_width, new_height), cv2.INTER_AREA)
    try:
        h,w,c = img.shape
    except:
        print('파일을 확인후 다시 시작하세요.')
        raise

    img_result=np.zeros((set_size,set_size, 3),dtype="uint8")
    # print(img_result.dtype,img.dtype)
    img_result[0:h,0:w]=img[:,:,:]
    ratio = h/H
    return img_result,ratio


file_list=glob("/content/cycle50/**/*.jpg", recursive=True)
# print(len(file_list))
for filename in file_list:
  img = cv2.imread(filename)
  label_name= filename.split("/")
  label_name[4]="라벨링데이터"
  label_name[-1]=label_name[-1].replace(".jpg",".json")
  label_name = "/".join(label_name)
  # print(label_name)
  with open(label_name, 'r') as f:

      json_data = json.load(f)

  bbox=json.dumps(json_data, ensure_ascii = False)
  bbox=json.loads(bbox)
  img,ratio = padding(img, 1280)
  h0,w0,c=img.shape
  crdnt_list = []
  for temp in bbox["annotations"]:
    x,y,w,h=int(temp["bbox"]["bndex_xcrdnt"]*ratio),int(temp["bbox"]["bndex_ycrdnt"]*ratio),int(temp["bbox"]["bndex_width"]*ratio),int(temp["bbox"]["bndex_hg"]*ratio)
    x,y=x+w//2,y+h//2
    crdnt_list.append([round(x/w0,6),round(y/h0,6),round(w/w0,6),round(h/h0,6)])

  tempbox=filename.split("/")[7].split("_")
  # print("label:",label_list.index(tempbox[0]+"_"+tempbox[1]))
  with open("/content/data/labels/"+filename.split("/")[-1].replace(".jpg",".txt"), 'w') as f:
    for temp in crdnt_list:
      f.writelines(str(label_list.index(tempbox[0]+"_"+tempbox[1]))+" "+" ".join(map(str, temp)))

  cv2.imwrite("/content/data/images/"+filename.split("/")[-1],img)


# for temp in crdnt_list:
#   print(temp)
#   img = cv2.rectangle(img,(int(w0*(temp[0]-temp[2]/2)),int(h0*(temp[1]-temp[3]/2))),(int(w0*(temp[0]+temp[2]/2)),int(h0*(temp[1]+temp[3]/2))),(0,0,0),10)
# plt.imshow(img)

In [10]:
# train/valid 데이터 분리
from sklearn.model_selection import train_test_split
from glob import glob
new_image_path = "/content/data/images"
images_path = list(glob(new_image_path+"/*"))

train_path_list,valid_path_list = train_test_split(images_path,test_size=0.1,random_state=42)

In [11]:
len(train_path_list),len(valid_path_list),len(train_path_list)+len(valid_path_list)

(1746, 195, 1941)

In [12]:
# 데이터 정보 txt 생성
with open('/content/data/train_dataset.txt', 'w') as f:
    f.writelines([f'{i}\n' for i in train_path_list])
with open('/content/data/valid_dataset.txt', 'w') as f:
    f.writelines([f'{i}\n' for i in valid_path_list]) 

In [13]:
# IPython에서 입력된 내용을 텍스트처럼 계속 사용하기 위한 register_line_cell_magic 정의
from IPython.core.magic import register_line_cell_magic

# register_line_cell_magic을 이용해 IPython에서 바로 사용할 수 있게 정의
# writetimplate를 쉽게 사용할 수 있음
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [14]:
%%writetemplate /content/data.yaml

train: '/content/data/train_dataset.txt'
val: '/content/data/valid_dataset.txt'

nc: len(label_list)
names: label_list

In [15]:
print(label_list)
len(label_list)

['전자레인지 사용_전자레인지', '석유풍로 사용_석유풍로', '썰기_상추', '다지기(고기)_고기', '튀기기_파프리카', '경고음_구급차', '차량 공회전_승용차', '깎기_감자', '따르기_포도주', '썰기_사과', '안마하기(전동)_전신 안마기', '엿장수_가위질', '튀기기_상추', '청소하기_무선 진공청소기', '횃불 지피기_기름 횃불', '키질_키', '따르기_막걸리', '확독 갈기_확독', '차량 공회전_버스', '떡메질_떡메', '키보드 치기_노트북', '굽기_닭고기', '믹서기 갈기_곡물갈기', '탄산 따르기_맥주', '커피콩 갈기_수동 그라인더', '탁구 치기_탁구', '숫돌 갈기_숫돌', '스프링클러 동작_스프링쿨러', '세탁기_통돌이 세탁기', '커피콩 갈기_자동 그라인더', '썰기_파', '선풍기 사용_공업용 선풍기', '교회 종_교회 종', '키보드 치기_일반 키보드', '싸리비 질_싸리비', '이발하기_전동 이발기', '커피 내리기_자판기', '튀기기_고기', '경고음_철길 건널목', '믹서기 갈기_고기갈기', '압력밥솥 사용_일반 압력밥솥', '콤바인 작업_수확', '청소하기_유선 진공청소기', '당구 치기_당구', '양수기 동작_펌프', '차량 주행_승용차', '경운기 작업_주행', '전 부치기_호박', '썰기_참외', '불 지피기_모닥불', '풍로 사용_풍로', '절구질_절구', '안마하기(전동)_핸드 안마기', '전 부치기_파', '차량 주행_버스', '썰기_배추', '무치기_겉절이', '믹서기 갈기_과일갈기', '세탁기_드럼 세탁기', '화투치기_화투 치기', '끓이기_찌개', '괘종시계 작동_괘종시계', '전 부치기_김치', '썰기_호박', '물레방아 돌기_물레방아', '커피 내리기_에스프레소 머신', '볼링 치기_볼링', '차량 주행_SUV', '다지기(채소)_양파', '면도하기(전동)_전기 면도기', '선풍기 사용_써큘레이터', '무치기_나물', '불 지피기_아궁이', '차량 공회전_1톤 트럭', '선박 운항_어선',

154

In [17]:
label_list[61]

'괘종시계 작동_괘종시계'

### YOLO 환경구성

In [16]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [17]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 14166, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 14166 (delta 64), reused 84 (delta 51), pack-reused 14060
Receiving objects: 100% (14166/14166), 13.54 MiB | 9.34 MiB/s, done.
Resolving deltas: 100% (9727/9727), done.


In [18]:
%cd /content/drive/MyDrive/yolov5

/content/drive/MyDrive/yolov5


In [20]:
!pip install -qr requirements.txt

In [20]:
import torch
import yaml
import cv2
from IPython.display import Image, clear_output

### 학습 시키기

In [18]:
%cd /content/drive/MyDrive/yolov5

/content/drive/MyDrive/yolov5


In [17]:
!pip install -qr requirements.txt

     |████████████████████████████████| 182 kB 10.7 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 1.6 MB 79.4 MB/s 


In [18]:
# 데이터 학습하기
!python train.py --img 1280 --batch 8 --epochs 100 --data /content/data.yaml --cfg ./models/yolov5s.yaml --weight '' --name image_detect --cache

train: weights=, cfg=./models/yolov5s.yaml, data=/content/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=8, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=image_detect, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
YOLOv5 🚀 v7.0-8-g350e8eb Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.

In [19]:
print("성공")

성공
